In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime

class PrcRcdProcesaOrigen():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
  def CargaMaestroClientes(self):    
          
    #Cursor
    cursor = self.connection.cursor()
    self.cursor = cursor
    
    ###### ------ Cliente Deudor ------ ######
    #seleccionamos todos los clientes 
    selectClienteDeudor = """
                          SELECT DISTINCT empresa.ID AS EMPRESA,
                          saldos.CUENTA_CMR AS CODIGO_DEUDOR_INTERNO,
                          formato.CODIGO_SBS AS CODIGO_DEUDOR_SBS,
                          ISNULL(formato.CODIGO_CIUU,'9999') AS CIIU,
                          NULL AS ZONA_REGISTRAL,
                          NULL AS OFICINA_REGISTRAL,
                          NULL AS TIPO_IDENTIFICACION_REGISTRAL,
                          REG_PUBLICO AS NUMERO_PARTIDA_REGISTRAL,
                          TIPO_DOC_TRIBUT AS TIPO_DOCUMENTO_TRIBUTARIO,
                          NUM_DOC_TRIBUT AS DOCUMENTO_TRIBUTARIO,
                          formato.TIPO_DOCUMENTO AS TIPO_DOCUMENTO_IDENTIDAD,
                          formato.NUM_DOCUMENTO AS DOCUMENTO_IDENTIDAD,
                          formato.TIPO_DOC_COMPLEMENTARIO AS TIPO_DOCUMENTO_IDENTIDAD_COMPLEMENTARIO,
                          formato.NUM_DOC_COMPLEMENTARIO AS DOCUMENTO_IDENTIDAD_COMPLEMENTARIO,
                          1 AS TIPO_PERSONA,
                          formato.RESIDENCIA,
                          formato.MAGNITUD_EMP AS MAGNITUD,
                          formato.ACCIONISTA,
                          NULL AS VINCULACION,
                          ISNULL(NULL,'0') AS RELACION_LABORAL,
                          ISNULL(formato.RESIDENCIA,'PE') AS PAIS_RESIDENCIA,
                          ISNULL(formato.PAIS, 'PE') AS PAIS_NACIMIENTO,
                          formato.SEXO AS GENERO,
                          formato.EST_CIVIL AS ESTADO_CIVIL,
                          formato.SIGLAS AS SIGLA,
                          formato.NOMBRE AS APELLIDO_PATERNO_RAZON_SOCIAL,
                          NULL AS APELLIDO_MATERNO,
                          NULL AS APELLIDO_CASADA,
                          NULL AS PRIMER_NOMBRE,
                          NULL AS SEGUNDO_NOMBRE,
                          NULL AS GRUPO_ECONOMICO,
                          formato.FECHA_NACIMIENTO AS FECHA_NACIMIENTO,
                          NULL AS OCUPACION_OFICIO_PROFESION,
                          NULL AS DESCRIPCION_OCUPACION,
                          NULL AS CARGO,
                          NULL AS DESCRIPCION_CARGO,
                          NULL AS DIRECCION,
                          NULL AS UBICACION_GEOGRAFICA,
                          NULL AS TELEFONO,
                          1 AS AGENCIA,
                          saldos.COD_CLIENTE,
                          formato.RUC,
                          formato.PFT,
                          formato.DFT,
                          formato.FILLER,
                          formato.TIP_TRIB,
                          formato.NUM_TRIB,
                          formato.TIP_OPE
          FROM [dbo].[BF_FORMATO_RCD] formato
          JOIN [dbo].[BF_RCD_SALDOS] saldos ON formato.NUM_TARJETA = saldos.CUENTA_CMR
          JOIN EMPRESA empresa ON empresa.CODIGO_EMPRESA_SBS =  CASE WHEN saldos.TIPO_CARTERA = 0  THEN '142' ELSE saldos.TIPO_CARTERA END 
          JOIN PRODUCTO producto ON producto.attribute1 = saldos.COD_PRODUCTO"""
    sql_selectClienteDeudor = pd.read_sql_query(selectClienteDeudor, self.connection)
    df_selectClienteDeudor = pd.DataFrame(sql_selectClienteDeudor)
    
    #Insertando datos de clientes nuevos
    insert = "INSERT INTO [dbo].[CLIENTE_DEUDOR] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    self.cursor.fast_executemany = True
    self.cursor.executemany(insert, df_selectClienteDeudor.values.tolist())
    self.connection.commit()    
    
    ###### ------ Cliente Deudor Producto ------ ######

    clienteProd = """SELECT DISTINCT clienteDeudor.ID AS CLIENTE_DEUDOR,
                      producto.ID AS PRODUCTO,
                      saldos.COD_OPERACION AS NUMERO_CUENTA,
                      NULL AS FECHA_APERTURA,
                      NULL AS FECHA_APROBACION,
                      NULL AS FECHA_CASTIGO,
                      0 AS DIAS_ATRASO_CASTIGO,
                      NULL AS FECHA_JUDICIAL,
                      NULL AS DIAS_ATRASO_JUDICIAL,
                      NULL AS FECHA_REFINANCIADO,
                      NULL AS DIAS_ATRASO_REFINANCIADO,
                      NULL AS FECHA_TRANSFERENCIA,
                      NULL AS DIAS_ATRASO_TRANSFERENCIA,
                      saldos.ESTADO_CUENTA
      FROM CLIENTE_DEUDOR clienteDeudor
      JOIN [BF_RCD_SALDOS] saldos ON clienteDeudor.CODIGO_DEUDOR_INTERNO = saldos.CUENTA_CMR
	  JOIN [BF_FORMATO_RCD] formato ON formato.NUM_TARJETA = saldos.CUENTA_CMR
      JOIN PRODUCTO producto ON producto.ATTRIBUTE1 = saldos.COD_PRODUCTO"""
    sql_clienteProd = pd.read_sql_query(clienteProd, self.connection)
    df_clienteProd = pd.DataFrame(sql_clienteProd)
    
    #cambiamos el tipo de dato según la BD
    #print(df_clienteProdExistentes.dtypes)
    df_clienteProd['PRODUCTO'] = df_clienteProd['PRODUCTO'].astype(int)
    df_clienteProd['DIAS_ATRASO_CASTIGO'] = df_clienteProd['DIAS_ATRASO_CASTIGO'].astype(int)
    #print(df_clienteProd.dtypes)
    
    #df_nuevosClientesProd = df_clienteProd.merge(df_clienteProdExistentes, indicator='i', how='outer').query('i == "left_only"').drop('i', 1)
    #print(df_nuevosClientesProd)
    #Insertando datos de clientes nuevos
    insert = "INSERT INTO [dbo].[CLIENTE_DEUDOR_PRODUCTO] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    self.cursor.fast_executemany = True
    self.cursor.executemany(insert, df_clienteProd.values.tolist())
    self.connection.commit()  
    
    self.cursor.close() 
    
  def CargaCliente(self):
    
    #Cursor
    cursor = self.connection.cursor()
    self.cursor = cursor
    
    #Reproceso #OBS: retirar new
    deleteClienteSaldos = """DELETE CPC
    FROM    CLIENTE C
            JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
            JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CPC.CLIENTE_PRODUCTO = CP.ID
    WHERE   FECHA_PROCESO =  """"'"+ self.fecha_proceso_str +"""'"""
    self.cursor.execute(deleteClienteSaldos)
    self.connection.commit()

    deleteClienteProducto = """DELETE CP
    FROM    CLIENTE C
            JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
    WHERE   FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    """
    self.cursor.execute(deleteClienteProducto)
    self.connection.commit()
    
    deleteCliente = ("DELETE C FROM CLIENTE C WHERE FECHA_PROCESO = '"+ self.fecha_proceso_str +"'")
    self.cursor.execute(deleteCliente)
    self.connection.commit()

    #Proceso
    ###### ------ Cliente ------ ######
    selectCliente = """SELECT DISTINCT clienteDeudor.ID AS CLIENTE_DEUDOR,
                    NULL AS SECUENCIA,
                    NULL AS CLASIFICACION_DEUDOR,
                    NULL AS CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO,
                    NULL AS CLASIFICACION_DEUDOR_CON_ALINEAMIENTO,
                    MAX(saldos.DIAS_MORA) AS DIAS_ATRASO,
                    1 AS INDICADOR_RIEGO_CAMBIARIO,
                    'A' AS INDICADOR_ATRASO,
                    NULL AS CLASIFICACION_DEUDOR_INTERNO,
                    ISNULL(se.VALOR,NULL) AS SOBREENDEUDAMIENTO
    FROM CLIENTE_DEUDOR clienteDeudor
    JOIN [BF_RCD_SALDOS] saldos ON clienteDeudor.CODIGO_DEUDOR_INTERNO = saldos.CUENTA_CMR
    LEFT JOIN BF_SOBREENDEUDAMIENTO se ON se.NRO_CUENTA_CMR = saldos.CUENTA_CMR
	GROUP BY clienteDeudor.ID, ISNULL(se.VALOR,NULL)"""
    sql_selectCliente = pd.read_sql_query(selectCliente, self.connection)
    df_selectCliente = pd.DataFrame(sql_selectCliente)
    
    #agregamos columna fecha de proceso
    df_selectCliente['FECHA_PROCESO'] = self.fecha_proceso_str
    
    #movemos la posicion de la columna fecha proceso al inicio del dataframe
    df_selectCliente = df_selectCliente.reindex(columns=['FECHA_PROCESO'] + list(df_selectCliente.columns[:-1])) 
    
    #agregamos secuencia
    df_selectCliente['SECUENCIA']=df_selectCliente.groupby('FECHA_PROCESO').cumcount() 
    #print(df_selectCliente.iloc[1])
    df_selectCliente = df_selectCliente.where(pd.notnull(df_selectCliente), None)
    
    #Insertando datos #OBS: quitar el new 
    insert = "INSERT INTO [dbo].[CLIENTE] VALUES (?,?,?,?,?,?,?,?,?,?,?)"
    self.cursor.fast_executemany = True
    self.cursor.executemany(insert, df_selectCliente.values.tolist())
    self.connection.commit()  
    
    ###### ------ Cliente Producto------ ######
    selectClienteProducto = """
                      SELECT DISTINCT cliente.ID AS CLIENTE,
                            clienteDeudorProducto.ID AS CLIENTE_DEUDOR_PRODUCTO,
                            CASE WHEN formato.TIPO_CLIENTE = 'B' THEN 1 ELSE 2 END AS TIPO_CLIENTE,
                            2 AS TIPO_CREDITO,
                            11 AS TIPO_CREDITO_SBS,
                            CASE WHEN saldos.DIAS_MORA>=180 THEN 'C' ELSE 'N' END AS SITUACION_CREDITO,
                            NULL AS CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO,
                            NULL AS CLASIFICACION_DEUDOR_INTERNA,
                            'N' AS AVAL,
                            saldos.DIAS_MORA,
                            NULL AS ALINEACION,
                            CASE WHEN saldos.DIAS_MORA>=180 THEN 'S' ELSE 'N' END AS INDICADOR_PAGO_CASTIGO,
                            0 AS SALDO_CAPITAL, --formato.SALDO AS SALDO_CAPITAL,
                            0 AS MONTO_DESEMBOLSADO, NULL --formato.MONTO_CREDITO AS MONTO_DESEMBOLSADO,
            FROM CLIENTE cliente
            JOIN CLIENTE_DEUDOR clienteDeudor ON cliente.CLIENTE_DEUDOR = clienteDeudor.ID
			JOIN CLIENTE_DEUDOR_PRODUCTO clienteDeudorProducto ON clienteDeudor.id = clienteDeudorProducto.CLIENTE_DEUDOR
			JOIN [BF_RCD_SALDOS] saldos ON cod_operacion = clienteDeudorProducto.NUMERO_CUENTA
			JOIN [BF_FORMATO_RCD] formato ON  formato.NUM_TARJETA  = saldos.CUENTA_CMR
            WHERE cliente.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
            """
    sql_selectClienteProducto = pd.read_sql_query(selectClienteProducto, self.connection)
    df_selectClienteProducto = pd.DataFrame(sql_selectClienteProducto)
    
    #Insertando datos #OBS: quitar el new 
    insert = "INSERT INTO [dbo].[CLIENTE_PRODUCTO] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    self.cursor.fast_executemany = True
    self.cursor.executemany(insert, df_selectClienteProducto.values.tolist())
    self.connection.commit() 
    
    self.cursor.close() 

#objeto = PrcRcdProcesaOrigen(4)
#objeto.CargaMaestroClientes()
#objeto.CargaCliente()   